<a href="https://colab.research.google.com/github/StrizhAXE/ML_in_busyness/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Расмотрим пример на датасете из репозитория UCI

Описание данных - https://archive.ics.uci.edu/ml/datasets/APS+Failure+at+Scania+Trucks#

In [1]:
import pandas as pd
import numpy as np
# import itertools

# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, log_loss, confusion_matrix

# import matplotlib.pyplot as plt
# import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

path  = 'd:\\GeekBrains\\Q4\\ML_in_busyness\\APS Failure at Scania Trucks Data Set\\'

In [2]:
data = pd.read_csv(path+"aps_failure_training_set.csv",index_col=False)
data_test = pd.read_csv(path+"aps_failure_test_set.csv",index_col=False)
data.head(3)

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,na,2130706438,280,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,neg,33058,na,0,na,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,neg,41040,na,228,100,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0


In [3]:
data['class'] = data['class'].map({'pos':1, 'neg':0})
data_test['class'] = data_test['class'].map({'pos':1, 'neg':0})
data.sample(5)

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
39045,0,80368,na,2130706432,898,0,0,0,0,0,...,608032,326848,627538,574502,454786,349076,583292,23076,0,0
25529,0,4130,0,20,20,0,0,0,0,0,...,12468,9492,20840,20468,36816,77964,282,0,0,0
45075,0,12,0,0,na,0,0,0,0,0,...,34,10,24,52,24,64,0,0,0,0
29406,0,10,0,0,0,0,0,0,0,0,...,46,8,32,16,70,82,0,0,0,0
23512,0,41450,na,122,88,0,0,0,0,0,...,525386,344790,854058,307922,2074,4,0,0,0,0


У нас есть 170 признаков и 1 целевая переменная (бинарная) - нужно определить какая деталь вышла из строя APS (pos) или нет (neg)

In [4]:
print(data.shape)

(60000, 171)


Всего 60000 деталей

Посмотрим на соотношение классов

In [5]:
data.iloc[:, 0].value_counts()

0    59000
1     1000
Name: class, dtype: int64

Обучаем модель catboost

In [6]:
x_train = data.iloc[:,1:]
y_train = data.iloc[:,0]
x_test =  data_test.iloc[:,1:]
y_test =  data_test.iloc[:,0]

In [7]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=5,
                           train_dir=path+'Catboost')

In [8]:
model.fit(x_train, y_train, verbose=False)
y_predict = model.predict(x_test)

Проверяем качество

In [9]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0))
    return  f1, roc, rec, prc

In [10]:
metrics_def = evaluate_results(np.array(y_test), np.array(y_predict))

Classification results:
f1: 79.88%
roc: 85.90%
recall: 72.00%
precision: 89.70%


In [12]:
metrics = {}
metrics.update({'metrics_def':(['f1', metrics_def[0]], ['roc', metrics_def[1]], ['recall', metrics_def[2]], ['precision_score', metrics_def[3]])})

### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [13]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,0].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.5 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 500/1000 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [14]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    59500
 1      500
Name: class_test, dtype: int64


* We now have just 59750 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

In [15]:
mod_data.sample(5)

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000,class_test
30680,1,669782,na,1024,316,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
54065,0,56354,na,180,180,0,0,0,0,0,...,246618,490668,487902,329826,201090,261062,3870,0,0,-1
37926,0,39094,na,2130706436,182,0,0,0,0,0,...,168104,435702,474560,304696,72162,36866,134,0,0,-1
39016,0,62196,na,76,76,0,0,0,0,0,...,357234,529548,250208,349634,369768,672,0,0,0,-1
27056,0,2840,0,2130706432,12,0,0,0,0,0,...,2962,10436,22176,32586,43442,174,0,0,0,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [16]:
x_data = mod_data.iloc[:,1:-1].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,0].values # original class

### 1. random negative sampling

In [17]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(500, 172) (500, 172)


In [18]:
model_mod = CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=5,
                           train_dir=path+'Catboost')

In [19]:
model_mod.fit(sample_train.iloc[:,1:-1].values, 
              sample_train.iloc[:,0].values,
              verbose=False)
y_predict = model_mod.predict(sample_test.iloc[:,1:-1].values)

In [20]:
metrics_mod = evaluate_results(np.array(sample_test.iloc[:,0].values), np.array(y_predict))

Classification results:
f1: 24.25%
roc: 94.89%
recall: 94.77%
precision: 13.91%


In [21]:
metrics.update({'metrics_mod':(['f1', metrics_mod[0]], ['roc', metrics_mod[1]], ['recall', metrics_mod[2]], ['precision_score', metrics_mod[3]])})


In [22]:
metrics.get('metrics_def'), metrics.get('metrics_mod')


((['f1', 0.7988165680473372],
  ['roc', 0.859008],
  ['recall', 0.72],
  ['precision_score', 0.8970099667774086]),
 (['f1', 0.24253347064881567],
  ['roc', 0.9489212594679919],
  ['recall', 0.9476861167002012],
  ['precision_score', 0.13906111603188662]))

При использовании random negative sampling лучше ROC и Recall, но при этом хуже precision и f1_score


### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)